In [4]:
import glob
import os
import sys
import random
import time
import numpy as np
import cv2

In [5]:
sys.path.append('E:\\projects\\carla\\WindowsNoEditor\\PythonAPI\\carla\\dist\\carla-0.9.9-py3.7-win-amd64.egg')

In [6]:
import carla

In [7]:
def process_img(image):
    img_flat = np.array(image.raw_data)
    img = img_flat.reshape(IM_HEIGHT, IM_WIDTH, 4)
    img = img[:, :, :3] # ignores alpha values (images have 4 channels: rgba)
    cv2.imshow('', img)
    cv2.waitKey(1)
    return img / 255.0

In [8]:
IM_WIDTH = 640
IM_HEIGHT = 480

In [9]:
actor_list = []

try:
    # set up client
    client = carla.Client('localhost', 2000)
    client.set_timeout(2.0)
    
    # set up vehicle
    world = client.get_world()
    blueprint_library = world.get_blueprint_library()
    bp = blueprint_library.filter('model3')[0]
    print(bp)
    
    spawn_point = random.choice(world.get_map().get_spawn_points())
    
    vehicle = world.spawn_actor(bp, spawn_point)
#     vehicle.set_autopilot(True)

    vehicle.apply_control(carla.VehicleControl(throttle=1.0, steer=0.0))
    actor_list.append(vehicle)
    
    # set up camera sensor
    cam_bp = blueprint_library.find('sensor.camera.rgb')
    cam_bp.set_attribute('image_size_x', f'{IM_WIDTH}')
    cam_bp.set_attribute('image_size_y', f'{IM_HEIGHT}')
    cam_bp.set_attribute('fov', '110')
    
    # place camera onto car
    spawn_point = carla.Transform(carla.Location(x=2.5, z=0.7))
    sensor = world.spawn_actor(cam_bp, spawn_point, attach_to=vehicle)
    actor_list.append(sensor)
    sensor.listen(lambda data: process_img(data))
finally:
    pass

ActorBlueprint(id=vehicle.tesla.model3,tags=[vehicle, tesla, model3])


In [16]:
# clean up
for actor in actor_list:
    actor.destroy()
print('All cleaned up!')

All cleaned up!


In [ ]:
cv2.destroyAllWindows()